In [3]:
# 인터파크 크롤링 역할
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--headless") # 브라우저창을 켜놓지 않은 상태로 값을 찾아옴
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=service,options=options)

url = "https://tickets.interpark.com/contents/genre/concert"
driver.get(url)
time.sleep(2)

try :
    hot_button = driver.find_element(By.XPATH, "//button[contains(text(),'요즘 HOT')]")
    print(hot_button)
    # hot_button.click()
    # time.sleep

except exception as e :
    print("요즘 HOT 버튼 클릭 실패 : ", e)

scroll_pause = 1
last_height = driver.execute_script("return document.body.scrollHeight") #return 자체가 종결 그래서 ; 없음

for _ in range(5):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height :
        break
    last_height = new_height

titles = driver.find_elements(By.CSS_SELECTOR,"li.TicketItem_goodsName__Ju76j")
dates = driver.find_elements(By.CSS_SELECTOR,"li.TicketItem_playDate__5ePr2")
places = driver.find_elements(By.CSS_SELECTOR,"li.TicketItem_placeName__ls_9C")
# print(titles, dates, places)
# 리스트가 3개인데 어떻게 할까?

data = []
for t, d, p in zip(titles, dates, places) : # 각각의 리스트를 동시출발
    data.append({
        "title": t.text.strip(),
        "date": d.text.strip(),
        "place": p.text.strip()
    }) # dictionary 로 해야 sql 들어갈때 행과 열로 잘 받을 수 있음
# print(data)
driver.quit()

df = pd.DataFrame(data)
df.head()

<selenium.webdriver.remote.webelement.WebElement (session="042772a887c18279db3558512e9f4030", element="f.C8C27EC22B1BB6196882C9C971EC156D.d.CDD9DE437C6C49D7D82B9D3DD194DC6C.e.44")>


,title,date,place
0,변진섭 전국투어 콘서트 : 변천사 시즌2,2025.02.09 -,전국 각 지역
1,RETRO MUSIC CONCERT 〈최재훈×플라워〉,"2025.08.09, 2025.10.19","서울,인천"
2,옥탑재즈,2025.02.14 - 2025.10.24,아트센터인천 다목적홀
3,2025 심수봉 전국투어 콘서트：꽃길 - 안동,2025.09.13,안동문화예술의전당 웅부홀
4,홍경민 매달리스트 콘서트,2025.08.02,SA HALL


In [18]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pymysql

conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'dPqls0415!',
    db = 'interpark',
    charset = 'utf8mb4'
)

cursor = conn.cursor()

for _, row in df.iterrows() :
    sql = """
    INSERT INTO performances (title, date_range, place) 
    VALUES (%s, %s, %s) 
    """
    cursor.execute(sql, (row["title"], row["date"], row["place"]))

conn.commit()
conn.close()